# Uncertainty calculation for model: SoilCarbon

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites, images and settings

In [4]:
# load the basic maps that needed for the analysis
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
biomeMask = biomeLayer.mask(biomeLayer.neq(98)).mask(biomeLayer.neq(99)).gt(0)
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000)
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)
# define the standard projection
stdProj = biomeLayer.projection()

# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialForestCover = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')

## 2 Load the soil carbon maps and apply the calculation

In [5]:
# load the carbon density layers
SandermannCarbonDiff = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU").unmask()
SandermannCarbonPresent = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_1km_Present").unmask()

# mask the diffrence layer
SandermannCarbonLoss = SandermannCarbonDiff.multiply(SandermannCarbonDiff.gt(0))

# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialCoverAdjusted.gte(0.1)

# calculate the sum of the potential in soil with the consideration of forest cover
SandermannCarbonStockLoss = SandermannCarbonLoss.multiply(pixelAreaMap).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

# SandermannCarbonStockLossResult =SandermannCarbonStockLoss.reduceRegion(reducer = ee.Reducer.sum(),
#                                                                         geometry = unboundedGeo,
#                                                                         crs = 'EPSG:4326',
#                                                                         crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                                         maxPixels = 1e9)

# print('Soil potential sum',SandermannCarbonStockLossResult.getInfo())

In [6]:
SandermannCarbonLossLower = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU_Lower").unmask()
SandermannCarbonLossUpper = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU_Upper").unmask()

SandermannCarbonStockLossLower = SandermannCarbonLossLower.multiply(pixelAreaMap).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)
SandermannCarbonStockLossUpper = SandermannCarbonLossUpper.multiply(pixelAreaMap).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

# SandermannCarbonStockLowerResult =SandermannCarbonStockLossLower.reduceRegion(reducer = ee.Reducer.sum(),
#                                                                         geometry = unboundedGeo,
#                                                                         crs = 'EPSG:4326',
#                                                                         crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                                         maxPixels = 1e9)

# print('Soil potential lower sum',SandermannCarbonStockLowerResult.getInfo())

# SandermannCarbonStockUpperResult =SandermannCarbonStockLossUpper.reduceRegion(reducer = ee.Reducer.sum(),
#                                                                         geometry = unboundedGeo,
#                                                                         crs = 'EPSG:4326',
#                                                                         crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                                         maxPixels = 1e9)

# print('Soil potential upper sum',SandermannCarbonStockUpperResult.getInfo())

## 3 Partioning the potential cover into different landuse types

In [7]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 3 Calculate the Upper and Lower of soil carbon potential

In [8]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = SandermannCarbonStockLoss.rename('SoilCarbon').addBands(SandermannCarbonStockLossLower.rename('SoilCarbonLower')).addBands(SandermannCarbonStockLossUpper.rename('SoilCarbonUpper'))

# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    perBiomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(perBiomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['SoilCarbon','SoilCarbonLower','SoilCarbonUpper'])#.round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/SoilCarbon_Uncertainty_for_diff_parts_at_Biome_Level.csv',header=True,mode='w+')
print(colored('The Soil carbon and potential uncertaintt partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The Soil carbon and potential uncertaintt partition results in biome: 



,SoilCarbon,SoilCarbonLower,SoilCarbonUpper
0,12.904557,6.449871,31.872728
1,1.936013,1.153363,3.080229
2,0.618024,0.304301,1.114871
3,15.119911,8.896084,28.705624
4,1.771906,0.662486,6.732126
5,1.448459,0.410863,27.185123
6,7.517337,4.558406,11.735743
7,3.732073,2.027256,6.807501
8,0.452222,0.294886,0.739886
9,0.816784,0.376503,1.755809


In [9]:
# stack the uncertainty maps to an image
intervalImage = ee.List([SandermannCarbonLoss,
                         SandermannCarbonLossLower,
                         SandermannCarbonLossUpper])

coversImage = maximumPotentialCover.rename('ConservationPotential').addBands(potentialForestCover.rename('AbsolutePotential')).addBands(freelandLeftMap.rename('FreelandPotential')).addBands(rangelandPotentialCover.rename('RangelandPotential')).addBands(pasturePotentialCover.rename('PasturePotential')).addBands(croplandPotentialCover.rename('CroplandPotential')).addBands(urbanPotentialCover.rename('UrbanPotential')).addBands(freelandForConsevation.rename('FreeToConservation'))

def landtypeUncertainPotential(perImage):
    absImage = coversImage.multiply(perImage).multiply(pixelAreaMap).multiply(potentialMask).multiply(biomeMask).divide(1000000000)
    output = absImage.reduceRegion(reducer= ee.Reducer.sum(),
                                   geometry= unboundedGeo,
                                   crs='EPSG:4326',
                                   crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                   maxPixels= 1e13)
    return output#.getInfo().get('Present')


statisticTable = intervalImage.map(landtypeUncertainPotential).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Types','ConservationPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
# allocate the row names into the 'Types' column
finalTablePotential = outputTable.assign(Types=['SoilCarbon',
                                                'SoilCarbonLower',
                                                'SoilCarbonUpper'])

# write the result into local folder 
finalTablePotential.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/SoilCarbon_Landuse_type_Uncertainty.csv',header=True,mode='w+')
print(colored('The Soil carbon uncertainty partition results in biome: \n', 'blue', attrs=['bold']))
# print the data frame of the results
finalTablePotential.head(3)

The Soil carbon uncertainty partition results in biome: 



,Types,ConservationPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,SoilCarbon,27.343209,49.305157,4.154445,3.073412,5.967752,8.486939,0.278854,3.394090
1,SoilCarbonLower,13.474602,26.828481,2.385280,1.883432,3.487478,5.425776,0.171845,1.610869
2,SoilCarbonUpper,91.084407,128.677078,10.522030,4.643033,9.367126,12.578619,0.456830,12.325133


In [65]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.